O objetivo dessa abordagem é substituir a forma como os embeddings de drogas e proteínas são gerados, trocando os modelos antigos por Transformers pré-treinados, que são muito mais poderosos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
import sys
import os

# 1. Montar o Google Drive
drive.mount('/content/drive')

# 2. Definir o caminho do projeto e mudar o diretório de trabalho
project_path = '/content/drive/MyDrive/Affinity2Vec_Collab'
os.chdir(project_path) # Usar os.chdir é mais robusto dentro de scripts

# 3. Adicionar o caminho do projeto ao sys.path
#    Esta é a parte que resolve o problema de importação!
if project_path not in sys.path:
  sys.path.append(project_path)

print(f"Diretório de trabalho atual: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Diretório de trabalho atual: /content/drive/MyDrive/Affinity2Vec_Collab


In [ ]:
# Instala as bibliotecas necessárias da Hugging Face e para deep learning
!pip install transformers torch sentencepiece

In [ ]:
input_folder = "Input/ChEMBL/"
drugFile = input_folder+"compoundsChEMBL34_SMILESgreaterThan5.tsv"
targetFile= input_folder+"targetsChEMBL34_noRepo3D.tsv"

## Gerar os Novos Embeddings de Proteínas com ESM-2

A seguinte função recebe uma sequência de aminoácidos e usa o modelo ESM-2 (Evolutionary Scale Modeling) da Meta AI para gerar um embedding de alta qualidade.

In [ ]:
import torch
from transformers import AutoTokenizer, EsmModel
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm # Para uma barra de progresso

# --- Função para gerar embeddings de proteínas ---
def get_protein_embedding(sequence, tokenizer, model, device):
    """Gera um embedding para uma única sequência de proteína usando ESM-2."""
    # Tokeniza a sequência
    inputs = tokenizer(sequence, return_tensors='pt', truncation=True, max_length=1022)

    # Mover os inputs para a GPU, se disponível
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Desativar o cálculo de gradiente para economizar memória e acelerar
    with torch.no_grad():
        # Passar a sequência pelo modelo
        outputs = model(**inputs)

    # Extrair os embeddings da última camada oculta
    # Usamos a média de todos os embeddings de tokens para obter um único vetor para a proteína inteira
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze()

    # Mover o embedding de volta para a CPU e converter para NumPy
    return embedding.cpu().numpy()

# --- Script principal para gerar todos os embeddings de proteínas ---
print("Configurando o modelo ESM-2...")
# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Carregar o tokenizador e o modelo ESM-2 pré-treinado
# 'esm2_t30_150M_UR50D' é um modelo de tamanho médio, bom para começar
tokenizer_prot = AutoTokenizer.from_pretrained("facebook/esm2_t30_150M_UR50D")
model_prot = EsmModel.from_pretrained("facebook/esm2_t30_150M_UR50D").to(device)

# Carregar seu arquivo de alvos (proteínas)
# ATENÇÃO: Monte seu Google Drive e ajuste o caminho se necessário
target_df = pd.read_csv("Input/ChEMBL/targetsChEMBL34_noRepo3D.tsv", sep='\t')

# A coluna com o ID do alvo é 'target' e a com a sequência é 'FASTA'
protein_sequences = target_df[['target', 'FASTA']].dropna().drop_duplicates(subset='target')

print(f"Gerando embeddings para {len(protein_sequences)} proteínas...")
protein_embeddings = {}
for index, row in tqdm(protein_sequences.iterrows(), total=len(protein_sequences)):
    seq_id = row['target']
    sequence = row['FASTA'] # Usar a coluna 'FASTA'
    try:
        embedding = get_protein_embedding(sequence, tokenizer_prot, model_prot, device)
        protein_embeddings[seq_id] = embedding
    except Exception as e:
        print(f"Erro ao processar a proteína {seq_id}: {e}")

# Converter o dicionário de embeddings para um DataFrame
protein_embedding_df = pd.DataFrame.from_dict(protein_embeddings, orient='index')
protein_embedding_df.index.name = 'target_id'

# Salvar os novos embeddings em um arquivo
protein_embedding_df.to_csv("EMBED/ChEMBL/Pr_ESM2_EMBED.tsv", sep='\t')
print("\nEmbeddings de proteínas salvos com sucesso!")


Configurando o modelo ESM-2...
Usando o dispositivo: cuda


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Gerando embeddings para 1512 proteínas...


  0%|          | 0/1512 [00:00<?, ?it/s]


Embeddings de proteínas salvos com sucesso!


## Gerar os Novos Embeddings de Drogas com ChemBERTa

O processo para as proteínas é muito similar para as drogas, mas usando um modelo treinado em moléculas (SMILES). Vamos usar o ChemBERTa.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from tqdm.notebook import tqdm

# --- Função para gerar embeddings de drogas ---
def get_drug_embedding(smiles, tokenizer, model, device):
    """Gera um embedding para uma única string SMILES usando ChemBERTa."""
    inputs = tokenizer(smiles, return_tensors='pt', truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Usar o embedding do token [CLS] (primeiro token) como representação da molécula
    embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding.cpu().numpy()

# --- Script principal para gerar todos os embeddings de drogas ---
print("\nConfigurando o modelo ChemBERTa...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Carregar o tokenizador e o modelo ChemBERTa pré-treinado
tokenizer_drug = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model_drug = AutoModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1").to(device)

# Carregar seu arquivo de compostos (drogas)
drug_df = pd.read_csv(drugFile, sep='\t')
drug_smiles = drug_df[['compound', 'SMILES']].dropna().drop_duplicates(subset='compound')

print(f"Gerando embeddings para {len(drug_smiles)} drogas...")
drug_embeddings = {}
for index, row in tqdm(drug_smiles.iterrows(), total=len(drug_smiles)):
    drug_id = row['compound']
    smiles = row['SMILES']
    try:
        embedding = get_drug_embedding(smiles, tokenizer_drug, model_drug, device)
        drug_embeddings[drug_id] = embedding
    except Exception as e:
        print(f"Erro ao processar a droga {drug_id}: {e}")

# Converter para DataFrame
drug_embedding_df = pd.DataFrame.from_dict(drug_embeddings, orient='index')
drug_embedding_df.index.name = 'drug_id'

# Salvar os novos embeddings
drug_embedding_df.to_csv("EMBED/ChEMBL/Dr_ChemBERTa_EMBED.tsv", sep='\t')
print("\nEmbeddings de drogas salvos com sucesso!")



Configurando o modelo ChemBERTa...
Usando o dispositivo: cuda
Gerando embeddings para 3340 drogas...


  0%|          | 0/3340 [00:00<?, ?it/s]


Embeddings de drogas salvos com sucesso!
